In [9]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import OrderedDict
from nltk.corpus import stopwords
from collections import Counter
import numpy as np
import pandas as pd
import math
import nltk
import operator
import json

In [14]:
# baixando stopwords em portugues

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/rpss/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rpss/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
# mock do array invertido
# segundo Ramom o indice sera composto por TERMO, DOCUMENTO, FREQUENCIA
# index = OrderedDict([
#     ('Blusa',[('DOCUMENTO1',2),('DOCUMENTO3',3)]),
#     ('Comprida',[('DOCUMENTO3',3),('DOCUMENTO4',1)]),
#     ('Longa',[('DOCUMENTO4',1),('DOCUMENTO1',1)]),
#     ('Cropped',[('DOCUMENTO4',1),('DOCUMENTO3',4)]),
#     ('Azul',[('DOCUMENTO3',2),('DOCUMENTO1',3)]),
#     ('Verde',[('DOCUMENTO4',1),('DOCUMENTO3',1)]),
# ])
index = json.load(open('./compress.json'), object_pairs_hook=OrderedDict)
consultaOriginal = "Blusas brancas e Longa Verde com Azul Azul"
len(index.items())
for term in index.items():
    print(term[0])

0
0024
0039
012
02
0362
04
05
074
0782
092
0px
1
10
100
12
12m
13
14
18
18m
1q
2
20
24m
2q
3
30
32
34
35
36
37
38
39
3m
3q
4
40
42
44
46
4q
5
50
6
6m
7
70
73
8
88
9
90
93
94
96
97
9m
abaixo
abe
aberta
aberto
abertura
abotoada
abotoamento
abus
acabamento
acessorio
acetinada
acetinado
acima
acinturada
acinturado
acinturar
acresent
adapta
aderir
adicionar
adora
adquirir
afina
agatha
agradavel
agua
ai
ainda
ajuda
ajusta
ajustada
akemi
alca
alcinha
alem
alfaiataria
alfaitaria
algod
algodao
aliada
aliado
alic
alonga
alongado
alongar
alta
alto
altura
amada
amamo
amarelo
amarraca
amarracao
amarraco
amarrado
amarrar
amassa
amelia
amigo
amor
ampla
ana
analic
andam
angela
anim
ano
apaixonant
aparec
aparent
apeca
aplicacao
aplicaco
apost
aposta
apostando
apostar
aproximadament
aqui
ar
areia
argola
arial
arras
arrasar
arrastao
arredondado
arrem
arrematando
arrematar
arrs
arrumadinho
asa
aspecto
assim
assimetica
assimetrica
assimetrico
ate
atempor
atencao
atitud
ato
atra
atraira
atual
autentica
aute

In [108]:
def consulting(text):
    My_stopwords = set(stopwords.words('portuguese'))
    palavras = word_tokenize(text)
    lista_Termos = [i for i in palavras if i not in My_stopwords]
    return lista_Termos

# exemplo de consulta do usuario
display(consulting('Blusas Longas de cor branca'))


['Blusas', 'Longas', 'cor', 'branca']

In [122]:
# precisamos saber quais Termos vou buscar dentro do grande index, serão apenas Termos que
# estão presentes na minha consulta, vamos então criar um array somente de termos que existem no
# meu discionario de termos

def postingTerms(fraseInput, index):
    indexSize = len(index)
    consulta = consulting(fraseInput)
#     print(consulta)
    pesquisa = []
#     display(fraseInput)
#     display(consulta)
    index.items()
#     for term, postings in index.items():
#             index[term] = compress_postings(postings)
    #falta trazet tudo pro min
    for fraseInput in consulta:
        for term in index.items():
            if term[0] in fraseInput:
                if term[0] not in pesquisa:
#                     print(term[0],"está dentro")
                    termo = str(term[0])
                    pesquisa.append(termo)
    print("pesquisa deve ter", pesquisa)
    return pesquisa

pesquisa = postingTerms(consultaOriginal, index)

# similarmente precisando saber quais Documentos vou buscar dentro do grande index

def documentNames(pesquisa,index):
    listaNomes =[]
    for doc in index.items():
        for termo in pesquisa:
            if termo in doc[0]:
                if doc[1][0][0] not in listaNomes:
                    listaNomes.append(doc[1][0][0])
#         print("documentos da pesquisa", listaNomes)
    return listaNomes
listaNomes = documentNames(pesquisa,index)

pesquisa deve ter ['l', 'u', 'usa', 'branca', 'c', 'g']


In [123]:
def matrixFrequencia(pesquisa,docs,index):
    matrix = np.zeros((len(pesquisa),len(docs)),dtype=object)
    #vamos criar um dataframe bonito que englobe os termos e os documentos da consulta 
    df = pd.DataFrame(matrix, columns=docs, index=pesquisa)
#     display("antes da analise",df)
    # Document-at-time olhando se o termo está dentro de um documento e por fim preenchendo os docs, sem trabalhar com frequência.
    for documento in docs:
        for termo in pesquisa:
#             print("documento", documento)
#             print("pesquisa", pesquisa)
            for post in index.items():
#                 print("termo",termo)
                if termo in post[0]:
#                     print("dale",documento)
#                     print("post", post[1][0][0])
                    if documento == post[1][0][0]:
#                         print("DEU BOM")
                        df.loc[termo,documento] =1
    display("depois analise",df)
    return df

def matrixFrequenciaTFIDF(pesquisa,docs,index):
    docs.insert(0, "000000010000000")
#     print(docs)
    matrix = np.zeros((len(pesquisa),len(docs)),dtype=object)
#     print(matrix)
    #vamos criar um dataframe bonito que englobe os termos e os documentos da consulta 
    df = pd.DataFrame(matrix, columns=docs, index=pesquisa)
#     display("antes da analise",df)
    
    # Document-at-time olhando se o termo está dentro de um documento e por fim preenchendo os docs, sem trabalhar com frequência.
    for documento in docs:
#         print(pesquisa)
        for termo in pesquisa:
            if termo in pesquisa and documento == "000000010000000":
                 #queremos levar em consideração a frequencia de pesquisa
                searchTimes = consultaOriginal.count(termo)
                df.loc[termo,documento] = searchTimes
            else:
                df.loc[termo,documento]=0
            for post in index.items():
                if termo in post[0]:
                    if documento == "000000010000000":
                        i=1
                    elif documento == post[1][0][0]:
#                         print("its a match")
                        if post[1][0][1] > 0:
                            df.loc[termo,documento] =post[1][0][1]
                        else:
                            df.loc[termo,documento] =0
    display("depois analise",df)
    return df

In [120]:
#exemplo de matrix Frequencia usando TFIDF
matrixFrequenciaTFIDF(pesquisa,listaNomes,index)

'depois analise'

,000000010000000,460,1738,220,1560,1580,587,1145,1613,1678,...,1571,691,690,1570,869,1679,383,881,747,1107
l,3,1,1,1,1,0,0,1,1,0,...,1,0,0,1,1,1,0,0,0,1
u,3,1,1,1,1,0,0,1,1,1,...,1,1,1,0,1,0,0,1,1,0
usa,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
branca,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c,2,1,1,1,1,2,1,1,1,1,...,0,0,0,0,0,1,1,1,1,1
g,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


,000000010000000,460,1738,220,1560,1580,587,1145,1613,1678,...,1571,691,690,1570,869,1679,383,881,747,1107
l,3,1,1,1,1,0,0,1,1,0,...,1,0,0,1,1,1,0,0,0,1
u,3,1,1,1,1,0,0,1,1,1,...,1,1,1,0,1,0,0,1,1,0
usa,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
branca,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c,2,1,1,1,1,2,1,1,1,1,...,0,0,0,0,0,1,1,1,1,1
g,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [88]:
#exemplo de matrix Frequencia SEM TFIDF
matrixFrequencia(pesquisa,listaNomes,index)

'depois analise'

,741,740,747,1623,460,8,1738,220,1560,1580,...,875,1162,299,690,869,383,881,1642,753,1107
u,0,1,1,0,1,0,1,1,1,0,...,0,1,0,1,1,0,1,0,0,0
l,1,1,0,1,1,1,1,1,1,0,...,0,1,0,0,1,0,0,0,0,1
usa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
i,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
m,1,1,1,0,1,1,1,1,1,0,...,1,0,0,0,0,0,0,0,0,0
om,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
p,1,1,0,1,1,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
bem,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
g,1,1,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
c,0,1,1,0,1,1,1,1,1,1,...,1,0,0,0,0,1,1,0,0,1


,741,740,747,1623,460,8,1738,220,1560,1580,...,875,1162,299,690,869,383,881,1642,753,1107
u,0,1,1,0,1,0,1,1,1,0,...,0,1,0,1,1,0,1,0,0,0
l,1,1,0,1,1,1,1,1,1,0,...,0,1,0,0,1,0,0,0,0,1
usa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
i,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
m,1,1,1,0,1,1,1,1,1,0,...,1,0,0,0,0,0,0,0,0,0
om,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
p,1,1,0,1,1,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
bem,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
g,1,1,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
c,0,1,1,0,1,1,1,1,1,1,...,1,0,0,0,0,1,1,0,0,1


In [125]:
# Similaridade Cosseno para medir o score
def cosine_similarity(v,w):
    similirity = np.dot(v,w)/(math.sqrt((np.dot(v,v))*(np.dot(w,w))))
#     display(similirity)
    return similirity

#trabalhando com respostas SEM
def scoreThis(matrix):
    documentsList = []    
    for docName in matrix.columns:  
        score = cosine_similarity(1, matrix[docName])
        score=max(score)
        documentsList.append((docName,score))
    documentsList.sort(key=operator.itemgetter(1),reverse=True)
    display(documentsList)
    return documentsList


def scoreThisTDIDF(matrix):
    documentsList = []
    timesList = matrix['000000010000000']
    for docName in matrix.columns:  
        score = cosine_similarity(timesList, matrix[docName])
        documentsList.append((docName,score))
    documentsList.sort(key=operator.itemgetter(1),reverse=True)
    return documentsList

In [116]:
# exemplo da pesquisa sem usar TFIDF
pesquisa = postingTerms("Quero Blusa Comprida bem Longa, Azul com cropped Verde", index)
listaNomes = documentNames(pesquisa,index)
matrix = matrixFrequencia(pesquisa,listaNomes,index)
scoreThis(matrix)

pesquisa deve ter ['u', 'l', 'usa', 'i', 'm', 'om', 'p', 'bem', 'g', 'c', 'crop', 'pe', 'pp']


'depois analise'

,741,740,747,1623,460,8,1738,220,1560,1580,...,875,1162,299,690,869,383,881,1642,753,1107
u,0,1,1,0,1,0,1,1,1,0,...,0,1,0,1,1,0,1,0,0,0
l,1,1,0,1,1,1,1,1,1,0,...,0,1,0,0,1,0,0,0,0,1
usa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
i,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
m,1,1,1,0,1,1,1,1,1,0,...,1,0,0,0,0,0,0,0,0,0
om,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
p,1,1,0,1,1,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
bem,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
g,1,1,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
c,0,1,1,0,1,1,1,1,1,1,...,1,0,0,0,0,1,1,0,0,1


[(122, 1.0),
 (1785, 1.0),
 (1572, 1.0),
 (585, 1.0),
 (1041, 1.0),
 (665, 1.0),
 (1581, 1.0),
 (1163, 1.0),
 (1171, 1.0),
 (86, 1.0),
 (298, 1.0),
 (1746, 1.0),
 (1708, 1.0),
 (758, 1.0),
 (1232, 1.0),
 (649, 1.0),
 (674, 1.0),
 (729, 1.0),
 (757, 1.0),
 (1314, 1.0),
 (1684, 1.0),
 (687, 1.0),
 (839, 1.0),
 (838, 1.0),
 (297, 1.0),
 (1094, 1.0),
 (1683, 1.0),
 (183, 1.0),
 (315, 1.0),
 (689, 1.0),
 (661, 1.0),
 (189, 1.0),
 (1562, 1.0),
 (307, 1.0),
 (728, 1.0),
 (878, 1.0),
 (1566, 1.0),
 (1488, 1.0),
 (1731, 1.0),
 (299, 1.0),
 (1642, 1.0),
 (753, 1.0),
 (1580, 0.7071067811865475),
 (1648, 0.7071067811865475),
 (1669, 0.7071067811865475),
 (376, 0.7071067811865475),
 (691, 0.7071067811865475),
 (1144, 0.7071067811865475),
 (1762, 0.7071067811865475),
 (887, 0.7071067811865475),
 (1091, 0.7071067811865475),
 (1134, 0.7071067811865475),
 (1169, 0.7071067811865475),
 (1044, 0.7071067811865475),
 (1043, 0.7071067811865475),
 (1093, 0.7071067811865475),
 (1105, 0.7071067811865475),
 (296

[(122, 1.0),
 (1785, 1.0),
 (1572, 1.0),
 (585, 1.0),
 (1041, 1.0),
 (665, 1.0),
 (1581, 1.0),
 (1163, 1.0),
 (1171, 1.0),
 (86, 1.0),
 (298, 1.0),
 (1746, 1.0),
 (1708, 1.0),
 (758, 1.0),
 (1232, 1.0),
 (649, 1.0),
 (674, 1.0),
 (729, 1.0),
 (757, 1.0),
 (1314, 1.0),
 (1684, 1.0),
 (687, 1.0),
 (839, 1.0),
 (838, 1.0),
 (297, 1.0),
 (1094, 1.0),
 (1683, 1.0),
 (183, 1.0),
 (315, 1.0),
 (689, 1.0),
 (661, 1.0),
 (189, 1.0),
 (1562, 1.0),
 (307, 1.0),
 (728, 1.0),
 (878, 1.0),
 (1566, 1.0),
 (1488, 1.0),
 (1731, 1.0),
 (299, 1.0),
 (1642, 1.0),
 (753, 1.0),
 (1580, 0.7071067811865475),
 (1648, 0.7071067811865475),
 (1669, 0.7071067811865475),
 (376, 0.7071067811865475),
 (691, 0.7071067811865475),
 (1144, 0.7071067811865475),
 (1762, 0.7071067811865475),
 (887, 0.7071067811865475),
 (1091, 0.7071067811865475),
 (1134, 0.7071067811865475),
 (1169, 0.7071067811865475),
 (1044, 0.7071067811865475),
 (1043, 0.7071067811865475),
 (1093, 0.7071067811865475),
 (1105, 0.7071067811865475),
 (296

In [126]:
# exemplo da pesquisa usando TFIDF
pesquisa = postingTerms("Quero Blusa Comprida bem Longa, Azul com cropped Verde", index)
listaNomes = documentNames(pesquisa,index)
matrix = matrixFrequenciaTFIDF(pesquisa,listaNomes,index)
scoreThisTDIDF(matrix)

pesquisa deve ter ['u', 'l', 'usa', 'i', 'm', 'om', 'p', 'bem', 'g', 'c', 'crop', 'pe', 'pp']


'depois analise'

,000000010000000,741,740,747,1623,460,8,1738,220,1560,...,875,1162,299,690,869,383,881,1642,753,1107
u,3,0,1,1,0,1,0,1,1,1,...,0,1,0,1,1,0,1,0,0,0
l,3,2,1,0,1,1,1,1,1,1,...,0,1,0,0,1,0,0,0,0,1
usa,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
i,0,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
m,1,1,1,1,0,1,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
om,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
p,0,1,1,0,1,1,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0
bem,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
g,1,2,1,0,0,0,1,0,1,0,...,0,0,0,0,1,0,0,0,0,0
c,2,0,1,1,0,1,1,1,1,1,...,1,0,0,0,0,1,1,0,0,1


[('000000010000000', 1.0),
 (1706, 0.9058216273156765),
 (1757, 0.8825226081218283),
 (14, 0.8153742483272113),
 (1738, 0.8006407690254357),
 (1567, 0.8006407690254357),
 (1763, 0.8006407690254357),
 (1575, 0.8006407690254357),
 (1113, 0.792593923901217),
 (703, 0.792593923901217),
 (648, 0.792593923901217),
 (1560, 0.7893522173763263),
 (1613, 0.7893522173763263),
 (1569, 0.7893522173763263),
 (18, 0.7893522173763263),
 (1554, 0.7893522173763263),
 (1645, 0.7893522173763263),
 (662, 0.7893522173763263),
 (1611, 0.7844645405527362),
 (991, 0.7844645405527362),
 (681, 0.7844645405527362),
 (1712, 0.7844645405527362),
 (386, 0.7844645405527362),
 (666, 0.7844645405527362),
 (701, 0.7844645405527362),
 (84, 0.7844645405527362),
 (1741, 0.7844645405527362),
 (718, 0.7844645405527362),
 (12, 0.76271276980969),
 (720, 0.76271276980969),
 (740, 0.7412493166611012),
 (220, 0.7412493166611012),
 (1145, 0.7412493166611012),
 (385, 0.7412493166611012),
 (1705, 0.7412493166611012),
 (1743, 0.74124